# Project Details

Our world is facing many urgent challenges, such as climate change, water insecurity, and food insecurity. Maintaining and improving quality of life around the world requires bringing together innovators across disciplines and countries to find creative solutions.

One critical tool for understanding and improving the urgent challenges facing our world is Earth observation data, meaning data that is gathered in outer space about life here on Earth! Earth observation data provides accurate and publicly accessible information on our atmosphere, oceans, ecosystems, land cover, and built environment. The United States and its partners have a long history of exploring outer space and making satellite, airborne, and in-situ sensor datasets openly available to all.

Your goal in this challenge is to create a visualization using Earth observation data that advances at least one of the following Sustainable Development Goals (SDGs):

- 6: Clean Water and Sanitation

By participating, you can be part of NASA's initiative to Transform to Open Science and to make Earth observation data available to all.

In [ ]:
# Operating System and File Handling
import os
import zipfile
import glob
import shutil

# Data Manipulation and Analysis
import pandas as pd
import numpy as np
import geopandas as gpd

# Visualization
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

# Web Scraping
from bs4 import BeautifulSoup

# Geometry and Spatial Analysis
from shapely.geometry import Point, Polygon, MultiPolygon

# Progress Bar
from tqdm.notebook import tqdm

# Image and Raster Data Processing
from io import BytesIO
import rasterio
import xarray as xr

In [ ]:
def open_geotiff_from_zip(zip_file_path, name_contains):
    # Open the provided zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        # Get a list of files in the zip archive that match the provided pattern
        matching_files = [file for file in zip_file.namelist() if name_contains in file]

        # Raise an error if no matching files are found
        if not matching_files:
            raise ValueError(f"No files in the zip archive match the pattern: {name_contains}")

        # Select the first matching file
        selected_file_name = matching_files[0]

        # Open the selected file within the zip archive
        with zip_file.open(selected_file_name) as file:
            # Check if the file is compressed and handle accordingly
            if selected_file_name.lower().endswith('.gz'):
                with gzip.GzipFile(fileobj=file) as gzipped_file:
                    # Read the content of the gzipped file and store it in a BytesIO object
                    geotiff_content = BytesIO(gzipped_file.read())
            else:
                # If the file is not compressed, read its content directly into a BytesIO object
                geotiff_content = BytesIO(file.read())

    # Open the GeoTIFF dataset using rasterio
    dataset = rasterio.open(geotiff_content)
    return dataset

In [ ]:
def calculate_water_indices(image_path):
    # Sentinel-2 L2A bands
    red_band = open_geotiff_from_zip(image_path, 'B04')  # Red, 664.6 nm (S2A), 665.0 nm (S2B)    
    green_band = open_geotiff_from_zip(image_path, 'B03')  # Green, 559.8 nm (S2A), 559.0 nm (S2B)    
    blue_band = open_geotiff_from_zip(image_path, 'B02')  # Blue, 492.4 nm (S2A), 492.1 nm (S2B)    
    nir_band = open_geotiff_from_zip(image_path, 'B08')  # NIR, 832.8 nm (S2A), 833.0 nm (S2B)    
    swir_band = open_geotiff_from_zip(image_path, 'B11')  # SWIR, 1613.7 nm (S2A), 1610.4 nm (S2B)
    swir_band_ = open_geotiff_from_zip(image_path, 'B12')  # SWIR, 2202.4 nm (S2A), 2185.7 nm (S2B)    
    water_vapour = open_geotiff_from_zip(image_path, 'B09')  # Water vapour, 945.1 nm (S2A), 943.2 nm (S2B)
    true_color = open_geotiff_from_zip(image_path, 'True_color')  # RGB Sat Image

    # Read band data as arrays
    red_data = red_band.read(1)
    green_data = green_band.read(1)
    blue_data = blue_band.read(1)
    nir_data = nir_band.read(1)
    swir_data = swir_band.read(1)

    # Read the true-color RGB bands
    true_color_ = true_color.read([1, 2, 3])
    # Stack the bands to create an RGB image
    true_color_satimage = true_color_.transpose(1, 2, 0)
    
    # Detect RGB channels
    band_descriptions = true_color.descriptions
    print('band_descriptions: ', band_descriptions)

    # Calculate Normalized Difference Water Index (NDWI) for water extent
    """
    NDWI (Normalized Difference Water Index):
        Measures water content.
        Normal: 0 to 0.2 (clear water).
        Dangerous: > 0.4 (increased turbidity).
    """
    ndwi = (green_data - nir_data) / (green_data + nir_data)
    # Apply NDWI threshold to identify water pixels
    water_mask = np.where(ndwi > 0.5, 1, 0)

    # Calculate the Normalized Water Index (NWI)
    nwi = (nir_data - swir_data) / (nir_data + swir_data)
    # Thresholding to identify water pixels
    threshold_w=0.4
    water_mask = np.where(nwi < threshold_w, 1, 0)
    # Get the pixel resolution
    transform = true_color.transform
    pixel_area = abs(transform[0]) * abs(transform[4])
    
    # Calculate Normalized Difference Snow Index (NDSI) for snow detection
    """
    NDSI (Normalized Difference Snow Index):
        Identifies snow cover.
        Normal: 0 to 0.2 (minimal snow).
        Dangerous: > 0.4 (extensive snow).
    """
    ndsi = (green_data - swir_data) / (green_data + swir_data)

    # Calculate Turbidity Index (e.g., Red-NIR or other suitable indices)
    """
    Turbidity Index:
        Gauges water clarity.
        Normal: 0 to 5 NTU (clear water).
        Dangerous: > 5 NTU (cloudy water).
    """
    turbidity_index = red_data / nir_data  # Placeholder, adjust as needed

    # Calculate Chlorophyll-a concentration using a suitable algorithm
    """
    Chlorophyll Concentration:
        Estimates chlorophyll levels.
        Normal: 0 to 10 µg/L (healthy ecosystems).
        Dangerous: > 20 µg/L (excessive algal growth).
        Unit: Micrograms per liter (µg/L)
    """
    # chlorophyll_concentration = 10 * (nir_data - red_data) / (nir_data + red_data)  # Placeholder, adjust as needed
    chlorophyll_concentration = np.where(water_mask, 10 * (nir_data - red_data) / (nir_data + red_data), 0)
    
    # Calculate Evapotranspiration Index (e.g., NDVI or other suitable indices)
    """
    Evapotranspiration Index:
        Reflects water loss from soil and plants.
        Normal: Varied based on vegetation health / 0 to 0.2 (Low vegetation cover, limited water loss)
        Moderate: 0.2 to 0.5 (Moderate vegetation cover, moderate water loss)
        Dangerous: 0.5 to 1 (High vegetation cover, significant water loss)
    """
    evapotranspiration_index = (nir_data - red_data) / (nir_data + red_data)  # Placeholder, adjust as needed

    # Calculate the Harmful Algal Bloom (HAB) Index based on Sentinel-2 satellite data.
    """
     Algal Bloom (AB) Index:
        Reflects the potential presence of harmful algal blooms in water bodies.
        Normal: Low likelihood of HABs / 0 to 0.1 (Low chlorophyll-a concentration)
        Moderate: 0.1 to 0.3 (Moderate chlorophyll-a concentration, increased risk)
        Dangerous: 0.3 to 1 (High chlorophyll-a concentration, significant risk of HABs)
    """

    # Calculate Algae Index (AI)
    algae_index = (1.5 * red_data - 1.3 * nir_data) / (red_data + 1.4 * blue_data - 0.6 * green_data + 0.04)

    # Create a DataFrame with median values
    median_values = {
        'NDWI': np.nanmedian(ndwi),
        'NDSI': np.nanmedian(ndsi),
        'Turbidity_Index': np.nanmedian(turbidity_index),
        'Chlorophyll_Concentration': np.nanmedian(chlorophyll_concentration),
        'Evapotranspiration_Index': np.nanmedian(evapotranspiration_index),
        'true_color_satimage': np.nanmean(true_color_satimage)
    }
    df_median_values = pd.DataFrame(median_values, index=[0])
    
    ndwi[np.isnan(ndwi)] = df_median_values.NDWI
    ndsi[np.isnan(ndsi)] = df_median_values.NDSI
    turbidity_index[np.isnan(turbidity_index)] = df_median_values.Turbidity_Index
    chlorophyll_concentration[np.isnan(chlorophyll_concentration)] = df_median_values.Chlorophyll_Concentration
    evapotranspiration_index[np.isnan(evapotranspiration_index)] = df_median_values.Evapotranspiration_Index
    true_color_satimage[np.isnan(true_color_satimage)] = df_median_values.true_color_satimage

    return  ndwi, ndsi, turbidity_index, chlorophyll_concentration, evapotranspiration_index, df_median_values, true_color_satimage

In [ ]:
def plot_indices(ndwi, turbidity_index, chlorophyll_concentration, evapotranspiration_index, true_color_satimage, title, fname, output='.'):
    # Create a figure with a specified size
    plt.figure(figsize=(14, 10))
    
    # Subplot 1: Normalized Difference Water Index (Water Extent)
    plt.subplot(3, 2, 1)
    plt.imshow(ndwi, cmap='Blues')
    plt.colorbar(label='NDWI')  # Add colorbar with label
    plt.title('Normalized Difference Water Index (Water Extent)')

    # Subplot 2: Normalized Difference Snow Index (snow detection)
    plt.subplot(3, 2, 2)
    plt.imshow(ndwi, cmap='plasma')
    plt.colorbar(label='NDSI')  # Add colorbar with label
    plt.title('Normalized Difference Snow Index (snow detection)')

    # Subplot 3: Turbidity Index
    plt.subplot(3, 2, 3)
    plt.imshow(turbidity_index, cmap='viridis')
    plt.colorbar(label='Turbidity Index')  # Add colorbar with label
    plt.title('Turbidity Index')

    # Subplot 4: Chlorophyll Concentration
    plt.subplot(3, 2, 4)
    plt.imshow(chlorophyll_concentration, cmap='viridis')
    plt.colorbar(label='Chlorophyll Concentration')  # Add colorbar with label
    plt.title('Chlorophyll Concentration')

    # Subplot 5: Evapotranspiration Index
    plt.subplot(3, 2, 5)
    plt.imshow(evapotranspiration_index, cmap='RdYlGn')
    plt.colorbar(label='Evapotranspiration Index')  # Add colorbar with label
    plt.title('Evapotranspiration Index')

    # Subplot 6: True Color Satellite Image
    plt.subplot(3, 2, 6)
    plt.imshow(true_color_satimage)
    plt.colorbar(label='True Color Satellite Image')  # Add colorbar with label
    plt.title('Satellite Image')

    # Add a big title to the entire set of subplots
    plt.suptitle('Water Analysis and Quality Indices: ' + title, fontsize=14, y=0.97)    

    # Save the figure as an image in the specified output directory
    plt.savefig(f'{output}/GIS_Analysis_Plot/{fname}.png')

    # Display the plot
    plt.show()

In [ ]:
def detect_water(image_path):
    # Sentinel-2 L2A bands
    green_band = open_geotiff_from_zip(image_path, 'B03')  # Green, 559.8 nm (S2A), 559.0 nm (S2B)	
    nir_band = open_geotiff_from_zip(image_path, 'B08')  # NIR, 832.8 nm (S2A), 833.0 nm (S2B)	
    
    # Extract bands for green and NIR (Near Infrared)
    green_data = green_band.read(1)
    nir_data = nir_band.read(1)

    # Calculate Normalized Difference Water Index (NDWI)
    ndwi = (green_data - nir_data) / (green_data + nir_data)

    # Thresholding to identify water pixels
    water_mask = ndwi >= 0.1  # Adjust the threshold as needed
    print()
    return water_mask

In [ ]:
# Define the source directory
src_dir_ = '.'
# Set the CDS API configuration file path
os.environ['CDSAPI_RC'] = f'{src_dir_}/.cdsapirc'

# Function to download weather data to DataFrame in parallel
def download_weather_data_to_dataframe_parallel(data):
    global src_dir_

    # List of weather variables to download
    variables = ['2m_temperature', 'total_precipitation', 'snowfall', 'wind_speed', 'mean_sea_level_pressure',
                 'relative_humidity', 'specific_humidity', 'total_cloud_cover', 'high_vegetation_cover',
                 'lake_total_layer_temperature', 'carbon_monoxide']

    # CDS API source and product type
    src = 'reanalysis-era5-single-levels'
    product_type = 'reanalysis'

    # Path to the zip file to store downloaded data
    zip_file_path = f'{src_dir_}/weather_data.zip'

    # Extract relevant information from the input data
    year, month, day, polygon, site_id = data['year'].values[0], data['month'].values[0], data['day'].values[0], data['geometry'].values[0], data['site_id'].values[0]

    # Adjusted temporary file name for storing the downloaded data
    temp_file_name = f'{src_dir_}/{site_id}_{year}_{month}_{day}.nc'

    # Check if the specified year is earlier than 1940, return an empty DataFrame
    if int(year) < 1940:
        return pd.DataFrame()

    # Use a zipfile to add downloaded files to a zip archive
    with zipfile.ZipFile(zip_file_path, 'a') as z:
        if os.path.basename(temp_file_name) not in z.namelist():
            # Calculate bounding box
            minx, miny, maxx, maxy = polygon
            area = [maxy, minx, miny, maxx]  # North, West, South, East
            # Initialize CDS API client
            c = cdsapi.Client(timeout=160, quiet=True, debug=False)

            try:
                # Download data from CDS API
                c.retrieve(
                    src,
                    {
                        'product_type': product_type,
                        'format': 'netcdf',
                        'variable': variables,
                        'year': year,
                        'month': month,
                        'day': day,
                        'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00',
                                 '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
                                 '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                        'area': area,
                    },
                    temp_file_name)

                # Add only the file to the main zip (not the entire path)
                z.write(temp_file_name, os.path.basename(temp_file_name))
                os.remove(temp_file_name)  # Remove the temporary file

            except Exception as e:
                print(f"Error in data retrieval: {e}")
                year -= 1

    # Adjust the temporary file name
    temp_file_name = f'{src_dir_}/{site_id}_{year}_{month}_{day}.nc'

    # Read the file from the main zip
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        if os.path.basename(temp_file_name) not in z.namelist():
            all_files = z.namelist()
            # Filter files to select only those ending with '.nc'
            nc_files = [file for file in all_files if file.startswith(f'{site_id}_{year}')]
            nc_files = [file for file in all_files if file.endswith(f'{year}_{month}_{day}_.nc')] if len(
                nc_files) == 0 else nc_files
            # Select a random file from the list
            random_file = random.choice(nc_files)
            # Adjust temp_file_name to the randomly selected file
            temp_file_name = random_file

        print('temp_file_name: ', temp_file_name)
        with z.open(os.path.basename(temp_file_name)) as f:
            with xr.open_dataset(f) as ds:
                # Convert to DataFrame
                df = ds.to_dataframe().reset_index()

                if df.empty:
                    raise ValueError("No data available for the specified date and area.")

                # Calculate statistics for each variable (min, max, mean)
                stats = df.describe().loc[['min', 'max', 'mean']]
                stats.drop(columns=['longitude', 'latitude'], inplace=True)

                # Create a DataFrame with statistics
                stats_df = stats.unstack().to_frame().transpose()
                stats_df.columns = [f'{var}_{stat}' for stat, var in stats_df.columns]
                stats_df['site_id'] = site_id
                stats_df['year'] = year
                stats_df['month'] = month

    return df

## Ganges_river Visualizations

In [ ]:
# Define the working directory for the Ganges River India
working_directory = './GIS/'
delivrables = 'deliverables/Ganges'

ganges_river_images = glob.glob(f'{working_directory}Rasters/Ganges_River/*.zip')
ganges_river_shape_fname = glob.glob(f'{working_directory}Shape_File/Ganges_River/*poly.shp')
ganges_river_images

In [ ]:
# Initialize empty lists to store shapes, area styles, and polygons
shapes = []
areastyle = []
poly = []

# Iterate over each shapefile in the list of Ganges river shapefiles
for file in ganges_river_shape_fname:
    # Read the shapefile and append it to the 'shapes' list
    shapes.append(gpd.read_file(file))
    # Define the area style for the shape (fill color and border color)
    areastyle.append({'fillColor': '#0BEE00', 'color': '#0BEE00'})

# Extract the first geometry from the first shape in the list and create a Polygon
poly = Polygon(shapes[0].geometry[0])
# Get the bounding box of the polygon's geometry
poly_geom = poly.bounds
# Get the centroid of the polygon
polycenter = poly.centroid

# Create a folium map centered at the polygon's centroid with a zoom level of 10
m = folium.Map([polycenter.y, polycenter.x], zoom_start=10)

# Iterate over each shape in the list and add it to the folium map
for shape in shapes:
    # Add GeoJSON layer to the map with the specified area style
    folium.GeoJson(shape, style_function=lambda x: areastyle[0]).add_to(m)

# Print the bounding box of the area covered by the map
print('area bounds : ', m.get_bounds())
# Get and set the map bounds to fit the area covered by the shapes
project_polygon_bounds = m.get_bounds()
m.fit_bounds(m.get_bounds())

# Display the folium map
display(m)

In [ ]:
ganges_water_df = pd.DataFrame()

# Iterate over Ganges river images
for images in tqdm(ganges_river_images):
    # Extract date information from the image filename
    f_name = os.path.basename(images)
    f_name = f_name.split('.')
    f_name_ = f_name[0][-8:len(f_name[0])]
    f_name = f_name[0][-6:len(f_name[0])]
    
    print(f'India_Ganges River Water Quality Analysis {f_name_}')
    
    # Detect water using a custom function
    water_mask = detect_water(images)    
    
    # Calculate water indices using a custom function
    ndwi, ndsi, turbidity_index, chlorophyll_concentration, evapotranspiration_index, ganges_river_2021_median_values_df, true_color_satimage = calculate_water_indices(images)
    
    # Add additional metadata to the DataFrame
    ganges_river_2021_median_values_df['Year'] = f_name[-4:len(f_name)]
    ganges_river_2021_median_values_df['Month'] = f_name[:2]
    ganges_river_2021_median_values_df['Day'] = f_name_[:2]
    ganges_river_2021_median_values_df['Location'] = 'Ganges_River_India'
    
    # Concatenate the calculated values to the main DataFrame
    ganges_water_df = pd.concat([ganges_water_df, ganges_river_2021_median_values_df], ignore_index=True)
    
    # Display the calculated water quality data
    display(ganges_river_2021_median_values_df)
    
    # Plot the water indices using a custom function
    plot_title = f_name_
    plot_indices(ndwi, turbidity_index, chlorophyll_concentration, evapotranspiration_index, true_color_satimage, plot_title + f"\n", plot_title, delivrables)
    
    # Create a dictionary containing weather data information
    weather_dict = {
        'year': f_name[-4:len(f_name)],
        'month': f_name[:2],
        'day': f_name_[:2],
        'geometry': [poly_geom],
        'site_id': 'Ganges_szone1'
    }

    # Download weather data and convert it to a DataFrame
    weather_data = pd.DataFrame(weather_dict)
    weather_df = download_weather_data_to_dataframe_parallel(weather_data)
    
    # Convert temperature from Kelvin to Celsius
    weather_df['t2m_C'] = weather_df['t2m'] - 273.15
    column_t2m_C = weather_df.pop('t2m_C')
    index_after = weather_df.columns.get_loc('t2m') + 1
    weather_df.insert(index_after, 't2m_C', column_t2m_C)

    # Standardize column names in the weather data
    std_cols = {
        't2m': '2m_temperature(K)',
        't2m_C': '2m_temperature(°C)',
        'tp': 'total_precipitation(m)',
        'sf': 'snowfall(m)',
        'msl': 'mean_sea_level_pressure(Pa)',
        'tcc': 'total_cloud_cover(0-1)',
        'cvh': 'high_vegetation_cover(0-1)',
        'ltlt': 'lake_total_layer_temperature(K)'
    }

    # Rename columns using the dictionary
    weather_df.rename(columns=std_cols, inplace=True)
    
    # Convert lake_total_layer_temperature from Kelvin to Celsius
    weather_df['lake_total_layer_temperature(°C)'] = weather_df['lake_total_layer_temperature(K)'] - 273.15
    
    # Convert 'time' column to datetime format
    weather_df['time'] = pd.to_datetime(weather_df['time'])
    # Extract the hour from the 'time' column
    weather_df['hour'] = weather_df['time'].dt.hour
    
    # Convert total_cloud_cover to float32
    weather_df['total_cloud_cover(0-1)'] = weather_df['total_cloud_cover(0-1)'].astype(np.float32)
    
    # Save weather data to a CSV file
    weather_df.to_csv(f'{delivrables}/Weather_dataframes/Ganges_weather_szone1_{plot_title}.csv', index=False)
    # Display the weather DataFrame
    display(weather_df)

# Save the Ganges water quality DataFrame to a CSV file
ganges_water_df.to_csv(f'{delivrables}/Ganges_water_quality_szone1_{min(ganges_water_df.Year)}-{max(ganges_water_df.Year)}.csv', index=False)

### ganges_river data_frames Analysis

In [ ]:
# Sort the ganges_water_df DataFrame by the 'Year' column in descending order
ganges_water_df = ganges_water_df.sort_values(by='Year', ascending=False)
ganges_water_df

## 3D interactive Turbidity_Index and Chlorophyll_Concentration Plot

In [ ]:
# Define a dictionary with water quality indices and their corresponding colors for the 3D plots
indexes_3D_plots = {'Turbidity_Index': 'orange', 'Chlorophyll_Concentration': 'red'}

# Extracting data
x = ganges_water_df['Year'].values
y = ganges_water_df['Month'].values

# Assigning different colors for each year
unique_years = ganges_water_df['Year'].unique()
colors = [f'hsl({hue}, 50%, 50%)' for hue in range(0, 360, int(360 / len(unique_years)))]

# Creating a dictionary to map years to colors
year_color_mapping = dict(zip(unique_years, colors))

# Iterate over each water quality index and its corresponding color
for index_, color_ in indexes_3D_plots.items():
    # Extract the z-axis data for the current index
    z = ganges_water_df[index_].values
    
    # Create a subplot with 3D scatter plot
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

    # Create a 3D scatter plot
    # Add a scatter plot for each year
    for year in unique_years:
        mask = ganges_water_df['Year'] == year
        trace = go.Scatter3d(
            x=x[mask], y=y[mask], z=z[mask],
            mode='markers',
            marker=dict(color=year_color_mapping[year]),
            name=str(year)
        )
        fig.add_trace(trace)
    
    # Update the layout of the 3D scatter plot
    fig.update_layout(
        scene=dict(aspectmode='cube'),
        scene_xaxis=dict(title='Year'),
        scene_yaxis=dict(title='Month'),
        scene_zaxis=dict(title=index_),
        title=f'3D Scatter Plot of {index_}',
        legend=dict(title='Year'),
        showlegend=True,
    )
        
    # Export the interactive visualization as an HTML file
    plot(fig, filename=f'{delivrables}/Water_Quality_Plots/3d_{index_}_scatter.html')

## WaterQuality Barplots

In [ ]:
# Define the selected columns for the pair plot
sel_cols = [
    'NDWI',
    'NDSI',
    'Turbidity_Index',
    'Chlorophyll_Concentration',
    'Evapotranspiration_Index',
    'Year'
]

# Create a DataFrame with selected columns
df = ganges_water_df[sel_cols].copy()

# Set the size of the plot
plt.figure(figsize=(14, 10))

# Create a pair plot with 'Year' as hue
sns.pairplot(df, hue='Year', palette='husl', markers="D")

# Set the title for the pair plot
plt.suptitle("Distribution and Pair-Plot of Water Quality Variables by Year (All Data)\n")

# Adjust layout to make more space for the title
plt.subplots_adjust(top=0.9)

# Save the figure to a PNG file
export_plot_fn = f'{delivrables}/Water_Quality_Plots/Water_Quality_pairplot.png'
plt.savefig(export_plot_fn)

## Weather data plot

In [ ]:
# Use glob to retrieve a list of file paths for CSV files in the specified directory
weather_df_files = glob.glob(f'{delivrables}/Weather_dataframes/*.csv')
weather_df_files

In [ ]:
# Iterate over each weather dataframe file
for file in tqdm(weather_df_files):
    # Define the columns to be used for plotting
    w_cols_ = [
        'hour',
        '2m_temperature(°C)',
        'total_precipitation(m)',
        'snowfall(m)',
        'mean_sea_level_pressure(Pa)',
        'high_vegetation_cover(0-1)',
    ]
    
    # Read the CSV file with selected columns
    file_name = os.path.basename(file)
    w_df = pd.read_csv(file, usecols=w_cols_)
    
    # Plotting
    # Create subplots with adjusted layout
    fig, axs = plt.subplots(w_df.shape[1]-1, 1, figsize=(14, 2 + 2 * (w_df.shape[1]-1)), sharex=True, gridspec_kw={'height_ratios': [2] * (w_df.shape[1]-1)})

    # Plot each column using seaborn
    for i, col in enumerate(w_df.columns[:-1]):
        ax = axs[i]
        sns.lineplot(data=w_df, x='hour', y=col, ax=ax, label=col, marker='o')

        # Alternate y-axis position for each subplot
        if i % 2 == 0:
            ax.yaxis.tick_left()
            ax.yaxis.set_label_position("left")
        else:
            ax.yaxis.tick_right()
            ax.yaxis.set_label_position("right")

        ax.set_ylabel(col)
    
    # Set common labels and title
    plt.xlabel('Hour')
    plt.yscale("log")
    plt.suptitle(f'Weather Data {file_name.split(".")[0]} (24 hours)')
    
    # Show legend
    plt.legend()

    # Save the figure to a PNG file
    plt.savefig(f'{delivrables}/Weather_Quality_Plots/{file_name.split(".")[0].split("_")[-1]}.png')

    # Show the plot
    plt.show()

## Dashboard Generation

In [ ]:
# File paths for the HTML visualizations
viz_turbidity_index = f'{delivrables}/Water_Quality_Plots/3d_Turbidity_Index_scatter.html'
viz_chlorophyll_concentration = f'{delivrables}/Water_Quality_Plots/3d_Chlorophyll_Concentration_scatter.html'

# Get data from 3D Turbidity Index visualization
with open(viz_turbidity_index, 'r', encoding='utf-8') as viz_turbidity_report:
    soup_turbidity = BeautifulSoup(viz_turbidity_report, 'html.parser')
viz_turbidity_content = soup_turbidity.find_all('div')
viz_turbidity_values = []

# Get the text content between the div tags
if viz_turbidity_content:
    # Extract the content of each script element
    for script in viz_turbidity_content:
        viz_turbidity_values.append(script)
else:
    print("Turbidity scripts not found.")
print('Turbidity scripts:', len(viz_turbidity_values))

# Get data from 3D Chlorophyll Concentration visualization
with open(viz_chlorophyll_concentration, 'r', encoding='utf-8') as viz_chlorophyll_report:
    soup_chlorophyll = BeautifulSoup(viz_chlorophyll_report, 'html.parser')
viz_chlorophyll_content = soup_chlorophyll.find_all('div')
viz_chlorophyll_values = []

# Get the text content between the div tags
if viz_chlorophyll_content:
    # Extract the content of each script element
    for script in viz_chlorophyll_content:
        viz_chlorophyll_values.append(script)
else:
    print("Chlorophyll scripts not found.")
print('Chlorophyll scripts:', len(viz_chlorophyll_values))

In [ ]:
project_location = 'Ganges'
project_name = 'Water Analysis and Data Visualization of Ganges River (India)'
output_fname = 'Water_Visualization_DashBoard.html'
area_bounds = project_polygon_bounds  # Use snake_case for variable names

data_list = glob.glob(f'{delivrables}/GIS_Analysis_Plot/*.png')
image_names = [os.path.basename(f).split('.')[0] for f in data_list]

# Define a custom sorting key function
def custom_sort(date_string):
    return (int(date_string[4:8]), int(date_string[2:4]), int(date_string[0:2]))

# Sort the list of dates using the custom key function
image_names_sorted = sorted(image_names, key=custom_sort)

# Print the sorted list
print('Sorted Image Names:', image_names_sorted)
print('Number of Images:', len(image_names_sorted))

In [ ]:
%%time

## Select the DashBoard template and the output path
viz_report_template = 'Water_Visualization_DashBoard_template.html'
viz_report_output = f'deliverables/{output_fname}'
shutil.copy(viz_report_template, viz_report_output)

# Modify the report template for the new report
with open(viz_report_output, 'r', encoding='utf-8') as viz_final_report:
    soup_final = BeautifulSoup(viz_final_report, 'html.parser')

# Selected main_script
script_check = soup_final.find('script', {'id': 'main_script'})
if script_check:
    script_check.string = script_check.string.replace('var areaBounds = ;', f'var areaBounds = {area_bounds};')
    script_check.string = script_check.string.replace('var imageNames = ;', f'var imageNames = {image_names_sorted};')
    script_check.string = script_check.string.replace('var project_location = ;', f'var project_location = "{project_location}";')
else:
    print("Target main_script section not found in the HTML file.")

# Change the Project Title
proj_title = soup_final.find('h1', {'id': 'project_title'})
if proj_title:
    proj_title.string = project_name
else:
    print("Target project_title section not found in the HTML file.")
    
# Change the slider maximum steps
slider_max = soup_final.find('input', {'id': 'image-slider'})
if slider_max:
    slider_max['max'] = len(image_names_sorted) - 1
else:
    print("Target image-slider section not found in the HTML file.")

# Chlorophyll_3d Interactive Plots
Chlorophyll_3d = soup_final.find('div', {'class': 'image-canvas-3D', 'id': 'Chlorophyll_3d'})
if Chlorophyll_3d:
    Chlorophyll_3d.insert_after(BeautifulSoup(' '.join(str(item) for item in viz_chlorophyll_values), 'html.parser'))
else:
    print("Target Chlorophyll_3d section not found in the HTML file.")

# Turbidity_3d Interactive Plots
Turbidity_3d = soup_final.find('div', {'class': 'image-canvas-3D', 'id': 'Turbidity_3d'})
if Turbidity_3d:
    Turbidity_3d.insert_after(BeautifulSoup(' '.join(str(item) for item in viz_turbidity_values), 'html.parser'))
else:
    print("Target Turbidity_3d section not found in the HTML file.")

# Save the modified HTML back to the file
with open(viz_report_output, 'w') as file:
    file.write(soup_final.prettify())